In [2]:
# conda install anaconda::openpyxl

import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
directory = "C:/Users/Daniel.DESKTOP-2H2HP2R/repos/hamburg_echarging/"

In [3]:
# Funktion zur Uwandlung von xlsx in csv
def convert_xls_to_csv(input_file, output_file, sheet_name=0):
    
    try:
        data = pd.read_excel(input_file, sheet_name=sheet_name)
        data.to_csv(output_file, index=False)
        print(f"Erfolgreich konvertiert: {input_file} -> {output_file}")
    except Exception as e:
        print(f"Fehler: {e}")

In [6]:
# Dateien
input_file = "dtv-dtvw-2004-2023-download-data.xlsx"
output_file = "dtv-dtvw-2004-2023-download-data.csv"

# Ausführen der Funktion
convert_xls_to_csv(input_file, output_file)

Erfolgreich konvertiert: dtv-dtvw-2004-2023-download-data.xlsx -> dtv-dtvw-2004-2023-download-data.csv


In [4]:
# Einladen des deutschlandweiten Ladesäulenregisters und der Hamburger Verkehrsstärke
ladesaeulenregister = pd.read_csv(directory +'Ladesaeulenregister_01122024.csv', header = 0, encoding= 'unicode_escape', delimiter= ';')
verkehrsstaerke = pd.read_csv(directory +'dtv-dtvw-2004-2023-download-data.csv', header = 0, encoding= 'unicode_escape', delimiter= ',')


C:\Users\Daniel.DESKTOP-2H2HP2R\AppData\Local\Temp\ipykernel_16524\3649124898.py:2: DtypeWarning: Columns (29,32) have mixed types. Specify dtype option on import or set low_memory=False.
  ladesaeulenregister = pd.read_csv(directory +'Ladesaeulenregister_01122024.csv', header = 0, encoding= 'unicode_escape', delimiter= ';')


In [14]:
# Informationenabfrag
print(verkehrsstaerke['zaehlstelle'].describe())
print(ladesaeulenregister['Betreiber'].describe())

count    1428.000000
mean     2924.792717
std       901.720521
min      1013.000000
25%      2392.000000
50%      2934.000000
75%      3556.000000
max      7621.000000
Name: zaehlstelle, dtype: float64
count                            80723
unique                           10849
top       EnBW mobility+ AG und Co.KG 
freq                              3972
Name: Betreiber, dtype: object


In [18]:
#Einladen Zaehlstellen CSV-Dateien
bedarfszaehlstellen = pd.read_csv(directory +'verkehrszaehlstellen_csv/app_kfz_bedarfszaehlstellen_EPSG_25832.csv', header = 0, encoding= 'unicode_escape', delimiter= ';')
dauerzaehlstellen = pd.read_csv(directory +'verkehrszaehlstellen_csv/app_kfz_dauerzaehlstellen_EPSG_25832.csv', header = 0, encoding= 'unicode_escape', delimiter= ';')
jahreszaehlstellen = pd.read_csv(directory +'verkehrszaehlstellen_csv/app_kfz_jaehrliche_zaehlstellen_EPSG_25832.csv', header = 0, encoding= 'unicode_escape', delimiter= ';')
pegel = pd.read_csv(directory +'verkehrszaehlstellen_csv/app_kfz_pegel_EPSG_25832.csv', header = 0, encoding= 'unicode_escape', delimiter= ';')

In [21]:
#Zusammenführen der CSV-Dateien
zaehlstellen_all = pd.concat([bedarfszaehlstellen, dauerzaehlstellen, jahreszaehlstellen, pegel], axis=0, ignore_index=True)
print(zaehlstellen_all)
zaehlstellen_all.to_csv("C:/Users/Daniel.DESKTOP-2H2HP2R/repos/hamburg_echarging/zaehlstellen_all.csv", index = False)

                                 ï»¿_fid  zaehlstelle  \
0     APP_KFZ_BEDARFSZAEHLSTELLEN_163670            2   
1     APP_KFZ_BEDARFSZAEHLSTELLEN_163671            3   
2     APP_KFZ_BEDARFSZAEHLSTELLEN_163672            5   
3     APP_KFZ_BEDARFSZAEHLSTELLEN_163673            6   
4     APP_KFZ_BEDARFSZAEHLSTELLEN_163674            7   
...                                  ...          ...   
2364                 APP_KFZ_PEGEL_37376         4088   
2365                 APP_KFZ_PEGEL_37377         4092   
2366                 APP_KFZ_PEGEL_37378         4094   
2367                 APP_KFZ_PEGEL_37379         4098   
2368                 APP_KFZ_PEGEL_37380         4102   

                                   bezeichnung  knoten letzte_zaehlung  \
0        Osdorfer LandstraÃe / WindmÃ¼hlenweg     NaN      01.01.2023   
1                Neue StraÃe / Harburger Ring     NaN      01.01.2023   
2     Elise-Averdieck-StraÃe / Oben Borgfelde     NaN      01.01.2024   
3          Oberhafe

In [24]:
# Zusammenfuehrung der georeferenzierten Zaehlstellen(zaehlstellen_all.csv) mit den Zaehlstaerken(dtv-dtvw-2004-2023-download-data.csv)
georef_verkehr = verkehrsstaerke.merge(
    zaehlstellen_all[['zaehlstelle', 'geom', 'coordinate_reference_system']],
    on='zaehlstelle',
    how='left'  # "left" sorgt dafür, dass alle Zeilen aus verkehrsstaerke erhalten bleiben
)

# Ergebnis speichern
georef_verkehr.to_csv('georef_verkehr.csv', index=False)

# Ergebnis anzeigen (optional)
print(georef_verkehr)

      zaehlstelle  Ebene                                     Bezeichnung  \
0            1013    371                    Krohnstieg NO Garstedter Weg   
1            1013    371                    Krohnstieg NO Garstedter Weg   
2            1013    371                    Krohnstieg NO Garstedter Weg   
3            1013    371                    Krohnstieg NO Garstedter Weg   
4            1017    372                    Garstedter Weg SW Krohnstieg   
...           ...    ...                                             ...   
1423         7459    221                    Hammer StraÃe NW Grenzknick   
1424         7621     96  EulenkrugstraÃe W Im Meienthun (Landesgrenze)   
1425         7621     96  EulenkrugstraÃe W Im Meienthun (Landesgrenze)   
1426         7621     96  EulenkrugstraÃe W Im Meienthun (Landesgrenze)   
1427         7621     96  EulenkrugstraÃe W Im Meienthun (Landesgrenze)   

                  Kategorie   2004   2005   2006   2007   2008   2009  ...  \
0        

In [8]:
# Ausschneiden des Ladesaeulenregisters auf die Stadflaeche Hamburgs und nur Schnellladesaeulen

# Filter anwenden: Nur Zeilen, in denen das Bundesland "Hamburg" ist
hamburg_tabelle = ladesaeulenregister[(ladesaeulenregister['Bundesland'] == 'Hamburg') & (ladesaeulenregister['Art der Ladeeinrichtung'] == 'Schnellladeeinrichtung')]

# Replace commas with dots in the coordinate columns (Latitude and Longitude)
hamburg_tabelle['Laengengrad'] = hamburg_tabelle['Laengengrad'].str.replace(',', '.')
hamburg_tabelle['Breitengrad'] = hamburg_tabelle['Breitengrad'].str.replace(',', '.')

# Gefilterte Tabelle speichern
hamburg_tabelle.to_csv('hamburg_tabelle.csv', index=False)

# Gefilterte Tabelle anzeigen (optional)
print(hamburg_tabelle)

                               Betreiber        Anzeigename (Karte)  \
36057     E.ON Drive Infrastructure GmbH  E.ON Drive Infrastructure   
36058     E.ON Drive Infrastructure GmbH  E.ON Drive Infrastructure   
36059     E.ON Drive Infrastructure GmbH  E.ON Drive Infrastructure   
36060     E.ON Drive Infrastructure GmbH  E.ON Drive Infrastructure   
36061     E.ON Drive Infrastructure GmbH  E.ON Drive Infrastructure   
...                                  ...                        ...   
37640             Shell Deutschland GmbH                        NaN   
37641             Shell Deutschland GmbH                        NaN   
37669  Hamburger Energiewerke Mobil GmbH                 HEnW-Mobil   
37670  Hamburger Energiewerke Mobil GmbH                 HEnW-Mobil   
37692                 Comfortcharge GmbH             Comfort Charge   

                Straï¿½e Hausnummer             Adresszusatz  Postleitzahl  \
36057   Mï¿½nckebergstr.          1  Media Markt, Parkdeck 7       20

C:\Users\Daniel.DESKTOP-2H2HP2R\AppData\Local\Temp\ipykernel_16524\1892423590.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hamburg_tabelle['Laengengrad'] = hamburg_tabelle['Laengengrad'].str.replace(',', '.')
C:\Users\Daniel.DESKTOP-2H2HP2R\AppData\Local\Temp\ipykernel_16524\1892423590.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hamburg_tabelle['Breitengrad'] = hamburg_tabelle['Breitengrad'].str.replace(',', '.')


In [9]:
# Kontrolle
print(hamburg_tabelle['Breitengrad'].describe())

count            330
unique           193
top       53.5716060
freq              12
Name: Breitengrad, dtype: object


In [22]:
# Kartenerstellung der Ladesaeulen und Verkehrsstaerken: https://geopandas.org/en/stable/gallery/plotting_with_folium.html

In [12]:
# Create point geometries
geometry = gpd.points_from_xy(hamburg_tabelle.Laengengrad, hamburg_tabelle.Breitengrad)
geo_df = gpd.GeoDataFrame(
    hamburg_tabelle,
    geometry=geometry
)
geo_df.head()

,Betreiber,Anzeigename (Karte),Straï¿½e,Hausnummer,Adresszusatz,Postleitzahl,Ort,Kreis/kreisfreie Stadt,Bundesland,Breitengrad,...,Steckertypen4,P4 [kW],Public Key4,Steckertypen5,P5 [kW],Public Key5,Steckertypen6,P6 [kW],Public Key6,geometry
36057,E.ON Drive Infrastructure GmbH,E.ON Drive Infrastructure,Mï¿½nckebergstr.,1,"Media Markt, Parkdeck 7",20095.0,Hamburg,Kreisfreie Stadt Hamburg,Hamburg,53.550522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (10.00572 53.55052)
36058,E.ON Drive Infrastructure GmbH,E.ON Drive Infrastructure,Mï¿½nckebergstr.,1,"Media Markt, Parkdeck 7",20095.0,Hamburg,Kreisfreie Stadt Hamburg,Hamburg,53.550522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (10.00572 53.55052)
36059,E.ON Drive Infrastructure GmbH,E.ON Drive Infrastructure,Mï¿½nckebergstr.,1,"Media Markt, Parkdeck 7",20095.0,Hamburg,Kreisfreie Stadt Hamburg,Hamburg,53.550522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (10.00572 53.55052)
36060,E.ON Drive Infrastructure GmbH,E.ON Drive Infrastructure,Mï¿½nckebergstr.,1,NaN,20095.0,Hamburg,Kreisfreie Stadt Hamburg,Hamburg,53.550522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (10.00572 53.55052)
36061,E.ON Drive Infrastructure GmbH,E.ON Drive Infrastructure,Mï¿½nckebergstr.,1,"Media Markt, Parkdeck 7",20095.0,Hamburg,Kreisfreie Stadt Hamburg,Hamburg,53.550522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (10.00572 53.55052)
